In [1]:
import psycopg2
from psycopg2 import OperationalError


def create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection

In [2]:
# Connection details
db_name = "project1"
db_user = "postgres"
db_password = "postgres"  # Update with your password
db_host = "pgdb"  # Update if your DB is hosted elsewhere
db_port = "5432"

# Create the connection
connection = create_connection(db_name, db_user, db_password, db_host, db_port)

Connection to PostgreSQL DB successful


In [3]:
try:
    # Create a new cursor object
    cursor = connection.cursor()

    # Define SQL statements for creating tables
    create_tables_queries = [
        '''
        CREATE TABLE IF NOT EXISTS DimHealthcost ( HealthcostID INTEGER PRIMARY KEY, "Healthcost Level" VARCHAR(255));

        ''',
        '''
        CREATE TABLE IF NOT EXISTS Dimhostee ( hostID INTEGER PRIMARY KEY, "hostbefore" VARCHAR(255));

        ''',
        '''
        CREATE TABLE IF NOT EXISTS Dimcountryarea ( country TEXT NOT NULL, region TEXT NOT NULL, countryID INTEGER PRIMARY KEY );

        ''',
        '''
        CREATE TABLE IF NOT EXISTS DimInternet ( internetID INTEGER PRIMARY KEY, "Internet Level" VARCHAR(255));

        ''',
        '''
        CREATE TABLE IF NOT EXISTS DimSport ( medelID INTEGER PRIMARY KEY, sportlevel VARCHAR(255));

        ''',
        '''
        CREATE TABLE IF NOT EXISTS FactHealth ( 
            countryID bigint PRIMARY KEY, 
            "health expenditure(GPT%)" NUMERIC,
            HealthcostID INTEGER, 
            InternetID INTEGER, 
            "Average Annual Growth Rate (%)" NUMERIC, 
            Healthdifference NUMERIC,
            "depressive disorders rate of Change" NUMERIC,
            medelID INTEGER, 
            hostID INTEGER, 
            "GDP(pc)" NUMERIC
        );


        ''',
        '''
        ALTER TABLE FactHealth ADD CONSTRAINT FK_countryID FOREIGN KEY (countryID) REFERENCES Dimcountryarea(countryID)
        ''',
        '''
        ALTER TABLE FactHealth ADD CONSTRAINT FK_HealthcostID FOREIGN KEY (HealthcostID) REFERENCES DimHealthcost(HealthcostID)
        ''',
        '''
        ALTER TABLE FactHealth ADD CONSTRAINT FK_InternetID FOREIGN KEY (InternetID) REFERENCES DimInternet(internetID)
        ''',
        '''
        ALTER TABLE FactHealth ADD CONSTRAINT FK_medelID FOREIGN KEY (medelID) REFERENCES DimSport(medelID)
        ''',
        '''
        ALTER TABLE FactHealth ADD CONSTRAINT FK_hostID FOREIGN KEY (hostID) REFERENCES Dimhostee(hostID)
        '''
    ]

    # Execute each create table SQL statement
    for query in create_tables_queries:
        cursor.execute(query)

    # Commit the changes to the database
    connection.commit()
    print("Tables 'dimensional tables' and 'Factables' created successfully.")
except Exception as e:
    print(f"An error occurred: {e}")


Tables 'dimensional tables' and 'Factables' created successfully.


In [4]:
def import_csv_to_table(connection, csv_file_path, table_name):
    try:
        cursor = connection.cursor()
        # Construct the COPY command
        copy_sql = f"""
        COPY {table_name} FROM stdin WITH CSV HEADER
        DELIMITER as ','
        """
        # Open the CSV file
        with open(csv_file_path, 'r') as f:
            # Execute the COPY command
            cursor.copy_expert(sql=copy_sql, file=f)
            connection.commit()
            print(f"Data from '{csv_file_path}' imported successfully into '{table_name}'.")
    except Exception as e:
        connection.rollback()  # Rollback on error
        print(f"Error: {e}")

In [5]:
# Path to your CSV file
csv_file_path = './projectdata/Client2/Dimregion.csv'
# Target table in your PostgreSQL database
table_name = 'Dimcountryarea'

# Import the CSV file into the PostgreSQL table
import_csv_to_table(connection, csv_file_path, table_name)

Data from './projectdata/Client2/Dimregion.csv' imported successfully into 'Dimcountryarea'.


In [6]:
# Path to your CSV file
csv_file_path = './projectdata/Client2/DimHealthcostGPT.csv'
# Target table in your PostgreSQL database
table_name = 'DimHealthcost'

# Import the CSV file into the PostgreSQL table
import_csv_to_table(connection, csv_file_path, table_name)


Data from './projectdata/Client2/DimHealthcostGPT.csv' imported successfully into 'DimHealthcost'.


In [7]:
# Path to your CSV file
csv_file_path = './projectdata/Client2/Dimhostee.csv'
# Target table in your PostgreSQL database
table_name = 'Dimhostee'

# Import the CSV file into the PostgreSQL table
import_csv_to_table(connection, csv_file_path, table_name)


Data from './projectdata/Client2/Dimhostee.csv' imported successfully into 'Dimhostee'.


In [8]:
# Path to your CSV file
csv_file_path = './projectdata/Client2/DimInternet.csv'
# Target table in your PostgreSQL database
table_name = 'DimInternet'

# Import the CSV file into the PostgreSQL table
import_csv_to_table(connection, csv_file_path, table_name)


Data from './projectdata/Client2/DimInternet.csv' imported successfully into 'DimInternet'.


In [9]:
# Path to your CSV file
csv_file_path = './projectdata/Client2/DimSport.csv'
# Target table in your PostgreSQL database
table_name = 'DimSport'

# Import the CSV file into the PostgreSQL table
import_csv_to_table(connection, csv_file_path, table_name)


Data from './projectdata/Client2/DimSport.csv' imported successfully into 'DimSport'.


In [10]:
# Path to your CSV file
csv_file_path = './projectdata/Client2/FactHealth.csv'
# Target table in your PostgreSQL database
table_name = 'FactHealth'

# Import the CSV file into the PostgreSQL table
import_csv_to_table(connection, csv_file_path, table_name)

Data from './projectdata/Client2/FactHealth.csv' imported successfully into 'FactHealth'.


In [11]:
cursor.close()
connection.close()